In [1]:
import os
import json
import threading
import datetime
 

def send_pii_request(request_id, base_payload_path, output_dir):
    """
    Sends a single PII request to the Azure Language API and saves the response.

    Args:
        request_id (int): The unique identifier for the request.
        base_payload_path (str): Path to the base JSON payload.
        output_dir (str): Directory where response files will be saved.
    """
    # Retrieve environment variables
    # LANGUAGE_ENDPOINT = os.getenv('LANGUAGE_ENDPOINT')
    # LANGUAGE_KEY = os.getenv('LANGUAGE_KEY')
    with open('input/secret.json', 'r', encoding='UTF-8') as file:
        secret_json = json.load(file)
    LANGUAGE_KEY = secret_json['LANGUAGE_KEY']
    LANGUAGE_ENDPOINT = secret_json['LANGUAGE_ENDPOINT']

    # if not LANGUAGE_ENDPOINT or not LANGUAGE_KEY:
    #     print(f"[ID {request_id}] Error: LANGUAGE_ENDPOINT and LANGUAGE_KEY must be set as environment variables.")
    #     return

    # Verify that the base payload file exists
    if not os.path.isfile(base_payload_path):
        print(f"[ID {request_id}] Error: Payload file '{base_payload_path}' does not exist.")
        return

    # Read the base payload
    try:
        with open(base_payload_path, 'r') as file:
            payload = json.load(file)
    except json.JSONDecodeError as e:
        print(f"[ID {request_id}] Error: Failed to parse JSON payload. {e}")
        return
    except Exception as e:
        print(f"[ID {request_id}] Error: An unexpected error occurred while reading the payload. {e}")
        return

    # Customize the payload with the current request ID
    try:
        payload['analysisInput']['documents'][0]['id'] = str(request_id)
        # Optionally, customize the text per request_id here
        # For example:
        # payload['analysisInput']['documents'][0]['text'] = f"Sample text for ID {request_id}."
    except (KeyError, IndexError) as e:
        print(f"[ID {request_id}] Error: Payload structure is incorrect. {e}")
        return

    # Save the customized payload to a temporary file
    customized_payload_path = f"test_pii_payload_{request_id}.json"
    try:
        with open(customized_payload_path, 'w') as file:
            json.dump(payload, file, indent=4)
    except Exception as e:
        print(f"[ID {request_id}] Error: Failed to write customized payload. {e}")
        return

    # Define the output response file path
    response_file = os.path.join(output_dir, f"pii{request_id}.json")

    # Construct the curl command
    curl_command = f'''
    curl -X POST "{LANGUAGE_ENDPOINT}/language/:analyze-text?api-version=2023-04-15-preview" \
    -H "Content-Type: application/json" \
    -H "Ocp-Apim-Subscription-Key: {LANGUAGE_KEY}" \
    -d "@{customized_payload_path}" > "{response_file}" 2>&1
    '''

    # Execute the curl command
    exit_code = os.system(curl_command)

    if exit_code != 0:
        print(f"[ID {request_id}] Warning: curl command exited with code {exit_code}. Check '{response_file}' for details.")
    else:
        print(f"[ID {request_id}] Success: Response saved to '{response_file}'.")

    # Optional: Retain the customized payload file or remove it
    # To remove:
    os.remove(customized_payload_path)

In [5]:
send_pii_request(request_id = 1, 
                 base_payload_path = "requests/test_pii_payload.json", 
                 output_dir = "requests_output/")

[ID 1] Error: Failed to parse JSON payload. Invalid \escape: line 12 column 671 (char 954)


In [6]:
import os

with open('input/secret.json', 'r', encoding='UTF-8') as file:
  secret_json = json.load(file)
  subscription_key = secret_json['LANGUAGE_KEY']
  endpoint = secret_json['LANGUAGE_ENDPOINT'] + "language/analyze-text/jobs?api-version=2023-04-15-preview"

# Prepare the JSON payload
json_payload = '''{
  "displayName": "Document PII Detection Task Example",
  "analysisInput": {
    "documents": [
      {
        "id": "1",
        "language": "en",
        "text": "Your text content here."
      }
    ]
  },
  "tasks": [
    {
      "kind": "PiiEntityRecognition",
      "taskName": "Document PII Detection Task 1",
      "parameters": {
        "model-version": "latest"
      }
    }
  ]
}'''

# Write the JSON payload to a temporary file
with open('payload.json', 'w') as file:
    file.write(json_payload)

# Define the curl command with output redirected to response.json
curl_command = f'''curl -X POST "{endpoint}" \
  -H "Content-Type: application/json" \
  -H "Ocp-Apim-Subscription-Key: {subscription_key}" \
  -d @payload.json > response.json'''

# Execute the curl command
exit_code = os.system(curl_command)

# Check if the command was successful
if exit_code != 0:
    print(f"curl command failed with exit code {exit_code}")
else:
    # Read and display the response
    with open('response.json', 'r') as response_file:
        response = response_file.read()
        print("API Response:")
        print(response)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

API Response:



100   375    0     0  100   375      0   1109 --:--:-- --:--:-- --:--:--  1109


In [7]:
import os
import json
import time

# # Replace with your actual subscription key
# subscription_key = "YOUR_SUBSCRIPTION_KEY"

# # Define the endpoint for submitting the job
# submit_endpoint = "https://azure-pii-detection.cognitiveservices.azure.com/language/analyze-text/jobs?api-version=2023-04-15-preview"

with open('input/secret.json', 'r', encoding='UTF-8') as file:
  secret_json = json.load(file)
  subscription_key = secret_json['LANGUAGE_KEY']
  submit_endpoint = secret_json['LANGUAGE_ENDPOINT'] + "language/analyze-text/jobs?api-version=2023-04-15-preview"

# Prepare the JSON payload
json_payload = '''{
  "displayName": "Document PII Detection Task Example",
  "analysisInput": {
    "documents": [
      {
        "id": "1",
        "language": "en",
        "text": "Call our office at 312-555-1234, or send an email to support@contoso.com"
      }
    ]
  },
  "tasks": [
    {
      "kind": "PiiEntityRecognition",
      "taskName": "Document PII Detection Task 1",
      "parameters": {
        "model-version": "latest"
      }
    }
  ]
}'''

# Write the JSON payload to a temporary file
with open('payload.json', 'w') as file:
    file.write(json_payload)

# Define the curl command to submit the job
curl_submit = f'''curl -X POST "{submit_endpoint}" \
  -H "Content-Type: application/json" \
  -H "Ocp-Apim-Subscription-Key: {subscription_key}" \
  -d @payload.json > submit_response.json'''

# Execute the curl command
exit_code = os.system(curl_submit)

# Check if the curl command was successful
if exit_code != 0:
    print("Failed to submit the job.")
    exit(1)

# Read the submission response
with open('submit_response.json', 'r') as f:
    try:
        submit_response = json.load(f)
    except json.JSONDecodeError:
        print("Failed to parse submission response. Please check submit_response.json for details.")
        exit(1)

# Extract jobId from the response
job_id = submit_response.get('jobId')
if not job_id:
    print("Job ID not found in the response.")
    print("Response:")
    print(json.dumps(submit_response, indent=2))
    exit(1)

print(f"Job submitted successfully. Job ID: {job_id}")

# Cleanup the payload file
os.remove('payload.json')


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

Failed to parse submission response. Please check submit_response.json for details.


100   424    0     0  100   424      0    305  0:00:01  0:00:01 --:--:--   305


NameError: name 'submit_response' is not defined